In [ ]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import pandas as pd
from web3 import Web3

from mainnet_launch.database.schema.full import (
    RebalancePlans,
    Destinations,
    DexSwapSteps,
    Tokens,
    RebalanceEvents,
)
from mainnet_launch.database.schema.postgres_operations import (
    get_full_table_as_orm,
    get_full_table_as_df,
    insert_avoid_conflicts,
    get_subset_not_already_in_column,
)
from mainnet_launch.data_fetching.get_state_by_block import get_raw_state_by_blocks, get_state_by_one_block
from mainnet_launch.data_fetching.tokemak_subgraph import fetch_autopool_rebalance_events_from_subgraph
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_transactions import (
    ensure_all_transactions_are_saved_in_db,
)
from mainnet_launch.constants import ALL_AUTOPOOLS, AutopoolConstants, time_decorator


from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_destinations_states_table import (
    build_lp_token_spot_and_safe_price_calls,
)


def _connect_plans_to_rebalance_evnets(
    rebalance_event_df: pd.DataFrame,
    rebalance_plan_df: pd.DataFrame,
) -> dict:
    rebalance_transaction_hash_to_rebalance_plan = {}

    for index in range(len(rebalance_event_df)):
        one_rebalance_event = rebalance_event_df.iloc[index]

        tx_hash = one_rebalance_event["transactionHash"]
        same_destinations = (rebalance_plan_df["token_out"] == one_rebalance_event["tokenOutAddress"]) & (
            rebalance_plan_df["token_in"] == one_rebalance_event["tokenInAddress"]
        )
        same_amount_out = rebalance_plan_df["amount_out"] == one_rebalance_event["tokenOutAmount"]

        window_start = one_rebalance_event["datetime_executed"] - pd.Timedelta(minutes=10)
        generated_no_more_than_one_hour_before = rebalance_plan_df["datetime_generated"].between(
            window_start, one_rebalance_event["datetime_executed"]
        )

        matches = rebalance_plan_df[same_destinations & same_amount_out & generated_no_more_than_one_hour_before]
        matches = matches.sort_values("datetime_generated", ascending=False).head(1)

        if matches.empty:
            rebalance_transaction_hash_to_rebalance_plan[tx_hash] = None
        else:
            rebalance_transaction_hash_to_rebalance_plan[tx_hash] = matches["file_name"].values[0]

    return rebalance_transaction_hash_to_rebalance_plan


def ensure_rebalance_events_are_updated():
    for autopool in ALL_AUTOPOOLS:

        rebalance_event_df = fetch_autopool_rebalance_events_from_subgraph(autopool)
        rebalance_plan_df = get_full_table_as_df(
            RebalancePlans,
            where_clause=(RebalancePlans.autopool_vault_address == autopool.autopool_eth_addr),
        )

        all_rebalance_event_hashes = rebalance_event_df["transactionHash"].to_list()

        rebalance_event_hashes_to_fetch = get_subset_not_already_in_column(
            RebalanceEvents,
            RebalanceEvents.tx_hash,
            all_rebalance_event_hashes,
            where_clause=RebalanceEvents.autopool_vault_address == autopool.autopool_eth_addr,
        )

        rebalance_event_df = rebalance_event_df[
            rebalance_event_df["transactionHash"].isin(rebalance_event_hashes_to_fetch)
        ].copy()

        if rebalance_event_df.empty:
            print(autopool.name, "no new rebalance events to fetch")
            continue

        hash_to_plan = _connect_plans_to_rebalance_evnets(rebalance_event_df, rebalance_plan_df)

        rebalance_event_df["rebalance_file_path"] = rebalance_event_df["transactionHash"].map(hash_to_plan)
        rebalance_event_df["autopool_vault_address"] = autopool.autopool_eth_addr
        rebalance_event_df["chain_id"] = autopool.chain.chain_id

        # ensure_all_transactions_are_saved_in_db(rebalance_event_df["transactionHash"].to_list(), autopool.chain)

        new_rebalance_event_rows = add_lp_token_safe_and_spot_prices(rebalance_event_df, autopool)
        # insert_avoid_conflicts(new_rebalance_event_rows, RebalanceEvents)


def add_lp_token_safe_and_spot_prices(
    rebalance_event_df: pd.DataFrame,
    autopool: AutopoolConstants,
    max_concurrent_fetches: int = 50,
) -> list[RebalanceEvents]:
    destinations_df = get_full_table_as_df(Destinations)
    destination_vault_address_to_pool = {
        d: p for d, p in zip(destinations_df["destination_vault_address"], destinations_df["pool"])
    }

    rebalance_event_df["poolInAddress"] = rebalance_event_df["destinationInAddress"].map(
        destination_vault_address_to_pool
    )
    rebalance_event_df["poolOutAddress"] = rebalance_event_df["destinationOutAddress"].map(
        destination_vault_address_to_pool
    )

    fetch_semaphore = threading.Semaphore(max_concurrent_fetches)

    def _fetch_prices_and_build_rebalance_event(row: pd.Series) -> RebalanceEvents:
        with fetch_semaphore:

            calls = build_lp_token_spot_and_safe_price_calls(
                destination_addresses=[row["destinationInAddress"], row["destinationOutAddress"]],
                lp_token_addresses=[row["tokenInAddress"], row["tokenOutAddress"]],
                pool_addresses=[row["poolInAddress"], row["poolOutAddress"]],
                chain=autopool.chain,
                base_asset=autopool.base_asset,
            )

            state = get_state_by_one_block(calls, int(row["blockNumber"]), chain=autopool.chain)

            if (autopool.autopool_eth_addr, "lp_token_spot_and_safe") in state:
                # the vault safe and spot prices are always 1.0
                state[(autopool.autopool_eth_addr, "lp_token_spot_and_safe")] = (1.0, 1.0)

            token_in_spot_value, token_in_safe_value = state[(row["destinationInAddress"], "lp_token_spot_and_safe")]
            token_out_spot_value, token_out_safe_value = state[(row["destinationOutAddress"], "lp_token_spot_and_safe")]

            safe_value_out = float(token_out_safe_value * row["tokenOutAmount"])
            safe_value_in = float(token_in_safe_value * row["tokenInAmount"])

            spot_value_in = float(token_in_spot_value * row["tokenInAmount"])
            spot_value_out = float(token_out_spot_value * row["tokenOutAmount"])

            return RebalanceEvents(
                tx_hash=row["transactionHash"],
                autopool_vault_address=row["autopool_vault_address"],
                chain_id=int(row["chain_id"]),
                rebalance_file_path=row["rebalance_file_path"],
                destination_out=row["destinationOutAddress"],
                destination_in=row["destinationInAddress"],
                quantity_out=float(row["tokenOutAmount"]),
                quantity_in=float(row["tokenInAmount"]),
                safe_value_out=safe_value_out,
                safe_value_in=safe_value_in,
                spot_value_in=spot_value_in,
                spot_value_out=spot_value_out,
                swap_offset_period=row["swapOffsetPeriod"],
            )

    new_rebalance_event_rows: list[RebalanceEvents] = []
    with ThreadPoolExecutor(max_workers=max_concurrent_fetches) as executor:
        future_to_idx = {
            executor.submit(_fetch_prices_and_build_rebalance_event, row): idx
            for idx, row in rebalance_event_df.iterrows()
        }
        for future in as_completed(future_to_idx):
            new_rebalance_event_rows.append(future.result())

    return new_rebalance_event_rows


ensure_rebalance_events_are_updated()

2025-05-29 11:32:39.703 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


2025-05-29 11:32:47,048 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-05-29 11:32:47,049 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-29 11:32:47,238 INFO sqlalchemy.engine.Engine select current_schema()
2025-05-29 11:32:47,240 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-29 11:32:47,442 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-05-29 11:32:47,443 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-29 11:32:47,639 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-29 11:32:47,640 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

,transactionHash,timestamp,blockNumber,autopool,tokenIn,destinationInAddress,tokenInAmount,tokenOut,destinationOutAddress,tokenOutAmount,tokenInAddress,tokenOutAddress,datetime_executed,swapOffsetPeriod,rebalance_file_path,autopool_vault_address,chain_id,poolInAddress,poolOutAddress
0,0xf4b0efb2e5d69ef7b7d000b4860a9d86b76013d9a042...,1726602143,20772483,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,99.961014,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,100.000000,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-17 19:42:23+00:00,50.0,rebalance_plan_1726602027_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
1,0xfdb95550d0720ff1f6ca2ff7331d1eb3194fe85ee8ab...,1726632143,20774967,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,314.380624,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,314.500371,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-18 04:02:23+00:00,50.0,rebalance_plan_1726632022_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
2,0xaa013ce865573532459a55487fb809c3e93edeed05af...,1726676555,20778643,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,144.676019,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,146.034135,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-18 16:22:35+00:00,50.0,rebalance_plan_1726676459_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
3,0x1e5ddbb3a2e56692c1c261b05b482298c1cd6bd6bcf3...,1726714667,20781802,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,133.437132,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,134.705433,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-19 02:57:47+00:00,50.0,rebalance_plan_1726714560_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
4,0xfb4645f276bf56d5ff8fef87301516ccb8ed0841a5db...,1726752383,20784928,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,335.615197,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,335.831303,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-19 13:26:23+00:00,50.0,rebalance_plan_1726752261_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,0x7356f89ae7b56795daec5ca6476ee2ce97f8124b784d...,1748255495,22566385,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,38.219307,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,38.175659,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x88794C65550DeB6b4087B7552eCf295113794410,2025-05-26 10:31:35+00:00,38.0,rebalance_plan_1748255424_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56

In [2]:
row = rebalance_event_df.iloc[0]

row

transactionHash           0xf4b0efb2e5d69ef7b7d000b4860a9d86b76013d9a042...
timestamp                                                        1726602143
blockNumber                                                        20772483
autopool                         0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
tokenIn                   {'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...
destinationInAddress             0xE382BBd32C4E202185762eA433278f4ED9E6151E
tokenInAmount                                                     99.961014
tokenOut                  {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...
destinationOutAddress            0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
tokenOutAmount                                                        100.0
tokenInAddress                   0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D
tokenOutAddress                  0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
datetime_executed                                 2024-09-17 19:42:23+00:00
swapOffsetPe

In [3]:
row.index

Index(['transactionHash', 'timestamp', 'blockNumber', 'autopool', 'tokenIn',
       'destinationInAddress', 'tokenInAmount', 'tokenOut',
       'destinationOutAddress', 'tokenOutAmount', 'tokenInAddress',
       'tokenOutAddress', 'datetime_executed', 'swapOffsetPeriod',
       'rebalance_file_path', 'autopool_vault_address', 'chain_id',
       'poolInAddress', 'poolOutAddress'],
      dtype='object')

In [ ]:
build_rebalance_event(row, ALL_AUTOPOOLS[0])

RebalanceEvents(tx_hash='0xf4b0efb2e5d69ef7b7d000b4860a9d86b76013d9a042e9753fe7c5e59e859b15', autopool_vault_address='0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56', chain_id=1, rebalance_file_path='rebalance_plan_1726602027_0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56.json', destination_out='0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56', destination_in='0xE382BBd32C4E202185762eA433278f4ED9E6151E', quantity_out=100.0, quantity_in=99.96101419887577, safe_value_out=100.0, safe_value_in=99.96300840200735, spot_value_in=99.9662607370716, spot_value_out=100.0, swap_offset_period=np.float64(50.0))